In [ ]:
# Install geospatial libraries (this takes 2-3 minutes)
%pip install -q earthengine-api geemap geopandas folium rasterio

print("Installation complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00
✅ Installation complete!


In [11]:
import ee
import geemap
import geopandas as gpd
import folium
from datetime import datetime
import os

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [10]:
import ee
import geemap


PROJECT_ID = 'asymmetric-moon-434905-r1'

print(f"🔧 Using project: {PROJECT_ID}")

try:
    # Try to initialize with your project
    ee.Initialize(project=PROJECT_ID)
    print("✅ Earth Engine initialized successfully!")

except:
    # First time - need to authenticate
    print("🔐 Authenticating Earth Engine...")
    print("   A browser window will open - sign in with Google")

    ee.Authenticate()

    # After authentication, initialize
    ee.Initialize(project=PROJECT_ID)
    print("✅ Authentication complete!")
    print("✅ Earth Engine ready to use!")

🔧 Using project: asymmetric-moon-434905-r1
✅ Earth Engine initialized successfully!


In [12]:
# Define Pauri Garhwal bounding box
min_lon, min_lat = 78.5, 29.8
max_lon, max_lat = 79.2, 30.3

# Create Earth Engine geometry
study_area = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Calculate center point for map display
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

print(f"📍 Study Area: Pauri Garhwal, Uttarakhand")
print(f"   Center: {center_lat:.2f}°N, {center_lon:.2f}°E")
print(f"   Area: ~{(max_lon-min_lon)*111:.0f} km × {(max_lat-min_lat)*111:.0f} km")

📍 Study Area: Pauri Garhwal, Uttarakhand
   Center: 30.05°N, 78.85°E
   Area: ~78 km × 56 km


In [22]:
# Create interactive map
Map = geemap.Map(center=[center_lat, center_lon], zoom=10)

# Add study area boundary
Map.addLayer(study_area, {'color': 'red'}, 'Study Area')

# Add basemap layers
Map.add_basemap('SATELLITE')

print("🗺️ Interactive map created! Scroll down to see it.")
Map

🗺️ Interactive map created! Scroll down to see it.


Map(center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [16]:
Map.close()

In [23]:
# Define date range (winter season - less clouds)
start_date = '2024-01-01'
end_date = '2024-03-31'

print(f"🛰️ Searching for Sentinel-2 images...")
print(f"   Date range: {start_date} to {end_date}")

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(study_area) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Less than 20% clouds

# Get the least cloudy image
image = s2.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Get image metadata
img_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
cloud_cover = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

print(f"✅ Found image from: {img_date}")
print(f"   Cloud cover: {cloud_cover:.1f}%")

# Select RGB bands for visualization
rgb_image = image.select(['B4', 'B3', 'B2'])  # Red, Green, Blue

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Add to map
Map.addLayer(rgb_image, vis_params, 'Sentinel-2 RGB')
Map

🛰️ Searching for Sentinel-2 images...
   Date range: 2024-01-01 to 2024-03-31
✅ Found image from: 2024-03-09
   Cloud cover: 0.0%


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [24]:
# Calculate NDVI
# Formula: (NIR - Red) / (NIR + Red)
nir = image.select('B8')  # Near-Infrared band
red = image.select('B4')  # Red band

ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

print("✅ NDVI calculated!")
print("   High values (0.6-0.9) = Dense vegetation (more fuel)")
print("   Low values (0-0.3) = Sparse vegetation")

# Visualization parameters for NDVI
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['brown', 'yellow', 'green', 'darkgreen']
}

# Add NDVI layer to map
Map.addLayer(ndvi, ndvi_vis, 'NDVI (Vegetation)')
Map

✅ NDVI calculated!
   High values (0.6-0.9) = Dense vegetation (more fuel)
   Low values (0-0.3) = Sparse vegetation


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [25]:
print("🏔️ Loading elevation data...")

# Load SRTM DEM (30m resolution)
dem = ee.Image('USGS/SRTMGL1_003').clip(study_area)

print("✅ DEM loaded!")
print("   Resolution: 30 meters")
print("   Source: NASA Shuttle Radar Topography Mission")

# Visualization parameters
dem_vis = {
    'min': 0,
    'max': 3000,
    'palette': ['green', 'yellow', 'brown', 'white']
}

# Add DEM layer
Map.addLayer(dem, dem_vis, 'Elevation (DEM)')
Map

🏔️ Loading elevation data...
✅ DEM loaded!
   Resolution: 30 meters
   Source: NASA Shuttle Radar Topography Mission


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [26]:
print("📐 Calculating slope...")

# Calculate slope in degrees
slope = ee.Terrain.slope(dem).rename('Slope')

print("✅ Slope calculated!")
print("   >30° = Very steep (fire spreads FAST)")
print("   10-30° = Moderate")
print("   <10° = Gentle slopes")

# Visualization
slope_vis = {
    'min': 0,
    'max': 45,
    'palette': ['green', 'yellow', 'orange', 'red']
}

Map.addLayer(slope, slope_vis, 'Slope (degrees)')
Map

📐 Calculating slope...
✅ Slope calculated!
   >30° = Very steep (fire spreads FAST)
   10-30° = Moderate
   <10° = Gentle slopes


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [27]:
print("🧭 Calculating aspect...")

# Calculate aspect (direction in degrees)
# 0° = North, 90° = East, 180° = South, 270° = West
aspect = ee.Terrain.aspect(dem).rename('Aspect')

print("✅ Aspect calculated!")
print("   South-facing (135-225°) = Highest fire risk (more sun)")
print("   North-facing (315-45°) = Lower risk (less sun)")

# Visualization (color-coded by direction)
aspect_vis = {
    'min': 0,
    'max': 360,
    'palette': ['blue', 'cyan', 'yellow', 'red', 'blue']
}

Map.addLayer(aspect, aspect_vis, 'Aspect (direction)')
Map

🧭 Calculating aspect...
✅ Aspect calculated!
   South-facing (135-225°) = Highest fire risk (more sun)
   North-facing (315-45°) = Lower risk (less sun)


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [28]:
print("🔥 Loading historical fire data...")

# Load MODIS fire product (2023-2024)
fires = ee.ImageCollection('FIRMS') \
    .filterBounds(study_area) \
    .filterDate('2023-01-01', '2024-10-16')

# Count number of fire detections
fire_count = fires.size().getInfo()

print(f"✅ Found {fire_count} fire hotspots in study area")
print("   Source: NASA FIRMS (Fire Information for Resource Management System)")

# Convert to points
fire_points = fires.map(lambda img: img.select('T21').gt(0))
fire_mosaic = fire_points.max()

# Visualization
fire_vis = {'palette': ['red']}
Map.addLayer(fire_mosaic, fire_vis, 'Historical Fires (2023-2024)')
Map

🔥 Loading historical fire data...
✅ Found 654 fire hotspots in study area
   Source: NASA FIRMS (Fire Information for Resource Management System)


Map(bottom=108412.0, center=[30.05, 78.85], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [29]:
print("💾 Preparing to export data to Google Drive...")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create export folder
export_folder = 'high_risk_prediction_data'
print(f"   Data will be saved to: MyDrive/{export_folder}/")

# Export NDVI
task_ndvi = ee.batch.Export.image.toDrive(
    image=ndvi,
    description='NDVI_Pauri_Garhwal',
    folder=export_folder,
    scale=10,  # 10m resolution
    region=study_area,
    maxPixels=1e13
)
task_ndvi.start()

# Export Slope
task_slope = ee.batch.Export.image.toDrive(
    image=slope,
    description='Slope_Pauri_Garhwal',
    folder=export_folder,
    scale=30,  # 30m resolution
    region=study_area,
    maxPixels=1e13
)
task_slope.start()

# Export Aspect
task_aspect = ee.batch.Export.image.toDrive(
    image=aspect,
    description='Aspect_Pauri_Garhwal',
    folder=export_folder,
    scale=30,
    region=study_area,
    maxPixels=1e13
)
task_aspect.start()

print("✅ Export tasks started!")
print("   Check your Google Drive in 5-10 minutes")
print("   Files will appear in: MyDrive/high_risk_prediction_data/")

💾 Preparing to export data to Google Drive...
Mounted at /content/drive
   Data will be saved to: MyDrive/high_risk_prediction_data/
✅ Export tasks started!
   Check your Google Drive in 5-10 minutes
   Files will appear in: MyDrive/high_risk_prediction_data/
